In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,721 kB]
Get:14 http://security.ubuntu.

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)


In [4]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView('sales')


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  select
    year(to_date(date,'yyyy-mm-dd')),
    round(avg(price),2)
  from sales
  where bedrooms=4
  group by year(to_date(date,'yyyy-mm-dd'))
  """).show(truncate=False)


+-------------------------------+--------------------+
|year(to_date(date, yyyy-mm-dd))|round(avg(price), 2)|
+-------------------------------+--------------------+
|2022                           |296363.88           |
|2019                           |300263.7            |
|2020                           |298353.78           |
|2021                           |301819.44           |
+-------------------------------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  select
    date_built,
    round(avg(price),2)
  from sales
  where bedrooms=3 and bathrooms=3
  group by date_built
  """).show(truncate=False)



+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|2016      |290555.07           |
|2012      |293683.19           |
|2017      |292676.79           |
|2014      |290852.27           |
|2013      |295962.27           |
|2011      |291117.47           |
|2015      |288770.3            |
|2010      |292859.62           |
+----------+--------------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  select
    date_built,
    round(avg(price),2)
  from sales
  where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000
  group by date_built
  """).show(truncate=False)


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|2016      |293965.1            |
|2012      |307539.97           |
|2017      |280317.58           |
|2014      |298264.72           |
|2013      |303676.79           |
|2011      |276553.81           |
|2015      |297609.97           |
|2010      |285010.22           |
+----------+--------------------+



In [24]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
  select
    view,
    round(avg(price),2) as avg
  from sales
  group by view
  having avg(price)>=350000
  order by view desc
  """).show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avg       |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 1.0451524257659912 seconds ---


In [26]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [27]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [28]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
  select
    view,
    round(avg(price),2) as avg
  from sales
  group by view
  having avg(price)>=350000
  order by view desc
  """).show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|avg       |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 1.2001819610595703 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [31]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet("delayed_partitioned")

In [32]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView("p_delays_p")

In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
  select
    view,
    round(avg(price),2) as avg
  from sales
  group by view
  having avg(price)>=350000
  order by view desc
  """).show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avg       |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 0.5527806282043457 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("sales")


False